<a href="https://colab.research.google.com/github/kobarmn/LangChain_GoogleColab/blob/main/test_OpenAPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# LangChain > Prompts を学ぶ。


#Prompts


#ライブラリをインストールする。
!pip install openai==0.28.0
!pip install langchain-community

import os
import openai

from langchain.chat_models import ChatOpenAI
from langchain.prompts import (
  ChatPromptTemplate,
  PromptTemplate,
  SystemMessagePromptTemplate,
  HumanMessagePromptTemplate

,
)
from langchain.schema import HumanMessage, SystemMessage

#環境変数 (APIキー)
openai.api_key = 'あなたのOpenAI.APIキーを選択してください。'


#Chat Models
chat = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0
)

#プロンプト作成
chat_prompt = ChatPromptTemplate.from_messages(
    [SystemMessagePromptTemplate.from_template("あなたは{prefectures}のツアーガイドです。"),
     HumanMessagePromptTemplate.from_template("以下の都道府県のおすすめの旅行計画を考えてください。\n\n都道府県名:{prefectures}")]
)

messages = chat_prompt.format_prompt(prefectures="東京都").to_messages()

#出力
result = chat(messages)
print(result.content)


In [ ]:
from pydantic import BaseModel, Field
from langchain.output_parsers import PydanticOutputParser

import os
import openai
from langchain.chat_models import ChatOpenAI
from langchain.prompts import (
  ChatPromptTemplate,
  PromptTemplate,
  SystemMessagePromptTemplate,
  HumanMessagePromptTemplate,
)
from langchain.schema import HumanMessage, SystemMessage

#環境変数 (APIキー)
openai.api_key = 'あなたのOpenAI.APIキーを選択してください。'

# 出力形式を指定するTourGuideクラス
class TourGuide(BaseModel):
    touristAttraction: list[str] = Field(description="都道府県の観光名所")
    steps: list[str] = Field(description="旅行計画のステップ")

# PydanticOutputParserに、TourGuideクラスを引き渡す。
parser = PydanticOutputParser(pydantic_object=TourGuide)
format_instructions = parser.get_format_instructions()


# PromptTemplate
template = """提示した都道府県における旅行計画を考えてください。

{format_instructions}

都道府県名:{prefectures}
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["prefectures"],
    partial_variables={"format_instructions": format_instructions}
)

# OpenAPI.LLMを呼び出す。
chat = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0,
)

formatted_prompt = prompt.format(prefectures="兵庫県")
messages=[HumanMessage(content=formatted_prompt)]
output = chat(messages)
print(output.content)

In [ ]:
#DocumentLoaderを学ぶ

!pip install GitPython==3.1.36
from langchain.document_loaders import GitLoader
from langchain.text_splitter import CharacterTextSplitter

def file_filter(file_path):
  return file_path.endswith(".mdx")

loader = GitLoader(
    clone_url="https://github.com/langchain-ai/langchain",
    repo_path="./langchain",
    branch="master",
    file_filter=file_filter,
)

raw_docs = loader.load()
print(raw_docs
)


text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

docs = text_splitter.split_documents(raw_docs)
print(docs)